# Fine-tune a Large Language Model with LoRA

This is a part of Lab 6 of the [EE292D Edge ML class](https://ee292d.github.io/) at Stanford, which covers parameter-efficient fine-tuning and deployment of LLMs.

You'll need a GPU for this exercise. As with previous labs, you can access them for free on Colab. [Click here](https://colab.research.google.com/github/ee292d/labs/blob/main/lab6/notebook.ipynb) to open this notebook in a Colab instance, then change your runtime type to GPU.

## Overview

Our goal is to fine-tune a small large language model (LLM) for a new task, then prepare it for deployment on a Raspberry Pi. In this example, we will fine-tune a base model that has been pre-trained for _completion_ (i.e., to predict the next words in the input sentence) so that we can use it for _chat_.

We're going to fine-tune using a technique called "low rank adaptation" (LoRA). Vanilla fine-tuning of LLMs requires a massive amount of GPU memory because we are directly updating the weights of the model during training. With LoRA, we train a small _adapter layer_ rather than retraining the whole model. Once we're done, we merge this small adapter layer into the original model to get our fine-tuned model.

To get started, install the required Python dependencies in your environment:

In [1]:
!pip install einops accelerate peft trl datasets transformers torch gguf protobuf sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 609.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)

## Choosing a Base Model

Since we want to deploy our fine-tuned model on a Raspberry Pi, we need to start with a small base model. In Lab 1, you experimented with Orca, which is a fine-tuned version of the 7 billion (7B) parameter Llama 2 model. For this lab, let's work with an even smaller model: [Phi-2](https://huggingface.co/microsoft/phi-2). At 2.7B parameters, Phi-2 can fit in about 5GB of RAM when loaded at 16-bit precision.

First, we'll get the model repo from HuggingFace. Run this cell to download the weights and load the model/tokenizer.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
   "microsoft/phi-2",
   torch_dtype=torch.bfloat16,
   trust_remote_code=True
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Now that we have the model loaded, we can try an input:


In [3]:
def prompt_phi(text):
    inputs = tokenizer(
        text,
        return_tensors="pt"
    )

    inputs.to("cuda")

    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.batch_decode(outputs)[0]

prompt_phi("Hello Phi!")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Hello Phi!\n\nI am writing to you today to share some exciting news about the Phi Beta Kappa Society. As you may know, Phi Beta Kappa is a prestigious academic honor society that recognizes and celebrates the achievements of outstanding students.\n\nIn the past, Phi Beta Kappa was only open to men, but in the 1960s, the society made a bold decision to become coeducational. This means that both men and women can now be a part of this amazing community. This was a big step towards equality and inclusivity in the academic world.\n\nIn the 1970s, Phi Beta Kappa continued to grow and expand. They started offering honorary membership to distinguished individuals who have made significant contributions to their fields. This was a way to recognize and honor those who have made a positive impact on society.\n\nIn the 1980s, Phi Beta Kappa made another important change. They started offering honorary membership to women, making it a truly coeducational society. This was a huge'

We want to chat with Phi-2, but at the moment it's just completing our sentences. We'll have to teach it some conversation skills with fine-tuning.

## Preparing a Fine-tuning Dataset

When we fine-tune a model, we're essentially showing it a large set of _training examples_ that we want the model's outputs to resemble. For example, if we wanted to tune a generative image model to produce images in the style of Garfield comics, we would fine-tune it on a large set of comics. By fine-tuning a model, we are nudging the weights so that the probability distribution of the output shifts toward the distribution observed in our set of training examples.

In this exercise, we want our model's outputs to look like a conversation between a human and a chatbot. Our fine-tuning dataset will accordingly consist of examples of conversations. There are a few key things we want the model to learn from these examples:

- **Structure:** We saw that our prompt to the base model earlier didn't result in a conversational response, but rather a rambling _completion_ of the input text. Our examples need to teach the model the structure of a conversation: a human and assistant taking turns responding to one another. We will demarcate this by `### Human:` and `### Assistant:` tags. That way, when we prompt the model with text like `### Human: Hey, how's it going today?` the model knows what comes next is a response, in the structure `### Assistant: ...`.

- **Tone:** Beyond structure, we are also teaching the model to adhere to a particular conversational tone. The tone and content of conversations in our training set will be reflected in the fine-tuned model's outputs. If we want the chatbot to take on a particular persona, or be conversant in a particular domain, we need to ensure our training set exemplifies that.

There are several datasets for chat, e.g., [OpenAssistant](https://huggingface.co/datasets/timdettmers/openassistant-guanaco), or [LMSys Chat](https://huggingface.co/datasets/lmsys/lmsys-chat-1m). Our Phi-2 base model is pre-trained on English text, so our fine-tuning set needs to be in English as well. For this exercise, we'll use a small English subset of the OpenAssistant dataset.

First, we'll load it and select a subset:

<a id='training_dataset'></a>

In [4]:
from datasets import load_dataset

dataset = load_dataset("sablo/oasst2_curated", split="train").select(range(200))
dataset

Generating train split:   0%|          | 0/4693 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/247 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 200
})

Next, we need to preprocess the data. Let's see what the raw format is.

In [5]:
example = dataset[0]
example["messages"]

[{'content': 'I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is there any way to salvage it?',
  'role': 'user'},
 {'content': "Yes, it's possible to fix runny mayonnaise! The most common reason for mayonnaise becoming runny is because the oil was added too quickly or the egg yolk wasn't emulsified properly. Here are some steps you can take to fix it:\n\n1. Separate another egg yolk and place it in a clean, dry bowl.\n2. Slowly add the runny mayonnaise to the egg yolk while whisking vigorously.\n3. Once all the runny mayonnaise has been added, continue whisking until the mixture has emulsified and thickened.\n4. If the mayonnaise is still too runny, you can add another egg yolk and repeat the process.\n\nIf the mayonnaise still won't thicken, you can try adding a small amount of dijon mustard or vinegar to the mixture, which can act as emulsifiers and help stabilize the mayonnaise. It's important to add these ingredients slowly and in s

Each item in the raw dataset is a list of dictionaries, where each dictionary is a turn in the conversation (`user` or `assistant`). We need to turn each item into a training example with our desired format.

In [6]:
def format_example(example):
    messages = example["messages"]
    training_instance = ""
    for turn in messages:
        if turn["role"] == "user":
            training_instance += f"### Human: {turn['content']}\n"
        elif turn["role"] == "assistant":
            training_instance += f"### Assistant: {turn['content']}\n"
    training_instance += tokenizer.eos_token
    return training_instance

print(format_example(example))

### Human: I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is there any way to salvage it?
### Assistant: Yes, it's possible to fix runny mayonnaise! The most common reason for mayonnaise becoming runny is because the oil was added too quickly or the egg yolk wasn't emulsified properly. Here are some steps you can take to fix it:

1. Separate another egg yolk and place it in a clean, dry bowl.
2. Slowly add the runny mayonnaise to the egg yolk while whisking vigorously.
3. Once all the runny mayonnaise has been added, continue whisking until the mixture has emulsified and thickened.
4. If the mayonnaise is still too runny, you can add another egg yolk and repeat the process.

If the mayonnaise still won't thicken, you can try adding a small amount of dijon mustard or vinegar to the mixture, which can act as emulsifiers and help stabilize the mayonnaise. It's important to add these ingredients slowly and in small amounts to avoid over-thi

Now, we'll map this function to the dataset to create a column of formatted training examples, then create an 80/20 train/test split for training.

In [7]:
dataset = dataset.map(lambda x: {"training_example": format_example(x)})
splits = dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Fine-tuning

To fine-tune with LoRA, we'll use the HuggingFace PEFT (Parameter Efficient Fine Tuning) library. This provides a convenient wrapper for LoRA, without requiring us to do any linear algebra :)

We're also going to train with [gradient checkpointing](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing) enabled. Though this makes training take a little longer, it further lowers the GPU memory requirement.

In [8]:
from transformers import TrainingArguments
from peft import get_peft_model, LoraConfig
from trl import SFTTrainer

output_dir = './phi-2-chat/'

model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    r=16,                   # the rank of the matrix we are training
    lora_alpha=8,           # lora scaling parameter
    lora_dropout=0.05,      # lora dropout probability
    bias="none",            # bias type
    task_type="CAUSAL_LM"   # type of model we are fine-tuning: a causal language model
)

# wrap the model for fine-tuning with lora
lora_model = get_peft_model(model, lora_config)

With the LoRA configuration out of the way, we're going to configure some standard training parameters. We'll wait to save the model until the last checkpoint, and we'll ensure the maximum sequence length in our training examples adheres to Phi-2's maximum (2048).

In [9]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_steps=20,
    save_steps=20,
    per_device_eval_batch_size=2,   # keep the batch size small so we don't run out of GPU memory
    per_device_train_batch_size=2,
    load_best_model_at_end=False
)

# SFT Trainer = supervised fine-tuning trainer = the interface for fine-tuning models provided by HuggingFace
trainer = SFTTrainer(
    model=lora_model,
    train_dataset=splits["train"],
    eval_dataset=splits["test"],
    peft_config=lora_config,
    dataset_text_field="training_example",
    max_seq_length=2048,            # the context window of phi-2 is 2048 tokens, so we set this as the max during fine-tuning
    tokenizer=tokenizer,
    args=training_args
)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Now we're ready to fine-tune the model. We're using a relatively small dataset, which should limit the training time to about 40 minutes. Note: if we weren't using gradient checkpointing, this would run much faster.

In [10]:
trainer.train()
trainer.save_model(f"{output_dir}/adapter-layer")

Step,Training Loss
20,1.658000
40,1.338300
60,1.438000
80,1.320800
100,1.400900
120,1.376100
140,1.351500
160,1.232600
180,1.466200
200,1.300800


You'll notice that the loss went down, but not as much as it could have. If we used a larger subset (or the full set) of training examples, loss would trend further downward. If you have time, consider [using a larger set of training examples](#training_dataset) from the dataset for training. If not, this amount of training is sufficient for our exercise.

Let's try a prompt to the fine-tuned model. We need to prepend the `### Human: ` tag, since this is the prompt structure that we fine-tuned the model on.

In [11]:
def prompt_phi_finetune(text):
    return prompt_phi(f"### Human: {text}\n")

print(prompt_phi_finetune("Hello Phi!"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### Human: Hello Phi!
### Assistant: Hello! How can I assist you today?
### Human: I am interested in learning more about the history of Phi.
### Assistant: Phi is a Greek letter that has been used in mathematics and science for centuries. It is often used to represent the golden ratio, which is a mathematical constant that appears in many natural phenomena, such as the spiral of a seashell or the branching of a tree.

The use of Phi in mathematics can be traced back to ancient Greece, where it was used to represent the ratio of the length of the hypotenuse of a right triangle to the length of its shorter leg. This ratio is approximately equal to 1.618, which is the value of the golden ratio.

In modern times, Phi is still used in mathematics and science, particularly in fields such as geometry, trigonometry, and calculus. It is also used in computer science and engineering, where it is used to represent


You'll notice two things:

1. Our outputs are now adhering to the turn-by-turn `### Human:` `### Assistant:` structure in our training examples!
2. The model continues the conversation past the first response.

To the second point, this is because we only fine-tuned the model on a small number of examples, so it hasn't seen enough data to learn when its response should stop. Even though this is the case, there is enough structure in the text to extract the chat responses. Let's do that now.

In [12]:
import re

def extract_response(text):
    """ We know where to extract the chat response since we fine-tuned
    the model to use the "### Assistant:"" tag. """
    pattern = r"### (Assistant|Phi):(.*?)\n"
    match = re.search(pattern, text)

    if match:
        return match.group(2).strip()
    else:
        print(f"No match for '{pattern}' found in '{text}'")
        return ""

def phi_chat(message):
    print(f"User: {message}")
    raw_response = prompt_phi_finetune(message)
    print(f"Phi: {extract_response(raw_response)}")

phi_chat("Hey Phi, how's it going?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: Hey Phi, how's it going?
Phi: Hello! I'm doing well, thank you. How can I assist you today?


If we want to create a multi-turn conversational interface, we simply chain together each turn in the conversation, providing Phi-2 the full "context" of the conversation each time we prompt it (up to its token limit of 2048). Our inputs should always have the same structure as our training examples. This is out of scope for this exercise, but you can imagine how we'd implement this. Give it a shot if you like!

## Merging Weights

Recall that LoRA trains an _adapter layer_, rather than a full model. If you look at the filesize of the weights we trained, this becomes apparent. While the Phi-2 model is about 5GB, the adapter that we trained is only about 30MB. In order to get the model ready for deployment on our Pi, we need to merge the adapter layer back into the weights of the base model. We'll do this now.

In [13]:
from transformers import AutoModelForCausalLM
import torch

base_model = AutoModelForCausalLM.from_pretrained(
   "microsoft/phi-2",
   torch_dtype=torch.bfloat16,
   trust_remote_code=True
).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from peft import PeftModel
import os
import shutil

adapter_dir = './phi-2-chat/adapter-layer'
output_dir = './phi-2-chat/merged-weights'

# this loads a base model with adapter layers attached.
# note: the first argument is a MODEL, while the second is a PATH TO A MODEL.
model = PeftModel.from_pretrained(
    base_model,
    adapter_dir
)

model = model.merge_and_unload() # merge adapters with the base model

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# save the merged weights. does not save any of the (important) tokenizer metadata...
model.save_pretrained(output_dir)

# ...so let's copy it over
for f in os.listdir(adapter_dir):
    if 'token' in f:
        print(f'copy {f} to {output_dir}')
        shutil.copyfile(os.path.join(adapter_dir, f), os.path.join(output_dir, f))

copy special_tokens_map.json to ./phi-2-chat/merged-weights
copy tokenizer.json to ./phi-2-chat/merged-weights
copy tokenizer_config.json to ./phi-2-chat/merged-weights
copy added_tokens.json to ./phi-2-chat/merged-weights


You've now fine-tuned a small base model for a new use case, and merged the weights for deployment/distribution. There's one step left to get them running on the Raspberry Pi, however: you'll need to convert the merged weights to the right format for the [llama.cpp](https://github.com/ggerganov/llama.cpp) framework. Let's get the framework source code, which includes the necessary conversion scripts.

In [15]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [16]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 22769, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 22769 (delta 0), reused 1 (delta 0), pack-reused 22764
Receiving objects: 100% (22769/22769), 27.12 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (16080/16080), done.


Now, we use llama.cpp's conversion scripts to convert our merged weights (in HuggingFace format) to llama.cpp's format (GGUF).

In [17]:
!python llama.cpp/convert-hf-to-gguf.py phi-2-chat/merged-weights --outfile phi-2-chat/phi-2-chat.gguf --outtype f16

Loading model: merged-weights
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
gguf: Adding 50000 merge(s).
gguf: Setting special token type bos to 50256
gguf: Setting special token type eos to 50256
gguf: Setting special token type unk to 50256
gguf: Setting special token type pad to 50256
Exporting model to 'phi-2-chat/phi-2-chat.gguf'
gguf: loading model part 'model-00001-of-00002.safetensors'
token_embd.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_norm.bias, n_dims = 1, torch.bfloat16 --> float32
blk.0.attn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_up.bias, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_up.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_down.bias, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_output.bia

You've now exported a version of the model that can run on your Raspberry Pi. Unfortunately it's large, over 5GB, so downloading it can be tricky. I recommend first copying it to your Google Drive, since that only takes a couple of minutes, and then you can download it from there.

To do that we'll mount your Google Drive as a folder on this Colab instance, which will require you to authenticate with your Google account.

In [18]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [19]:
!cp phi-2-chat/phi-2-chat.gguf '/content/gdrive/My Drive/'

In [20]:
!ls -lt '/content/gdrive/My Drive/phi-2-chat.gguf'

-rw------- 1 root root 5563095584 Apr 22 17:10 '/content/gdrive/My Drive/phi-2-chat.gguf'


You should now be able to download the file from Drive, and continue with the rest of [the Lab 6 tutorial](https://github.com/ee292d/labs/blob/main/lab6/README.md#deploying).